In [1]:
# !pip install repeng accelerate datasets vllm

In [2]:
#!/usr/bin/env python3
import os
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from tqdm.auto import tqdm

# ─── Config ────────────────────────────────────────────────────────────────
MODEL_NAME       = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
OUTPUT_PATH      = "gsm8k_chains.pkl"
CHECKPOINT_DIR   = "checkpoints"
CHECKPOINT_EVERY = 100
MAX_CHAIN_TOKENS = 4096

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ─── Initialize vLLM + tokenizer ──────────────────────────────────────────
llm = LLM(
    model=MODEL_NAME,
    tensor_parallel_size=1,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# ─── Generation parameters ────────────────────────────────────────────────
sampling_params = SamplingParams(
    temperature=0.6,
    max_tokens=MAX_CHAIN_TOKENS,
    top_p=0.95,
)

# ─── Prepare the dataset ──────────────────────────────────────────────────
num_problems = 1200
gsm8k_full = load_dataset("gsm8k", "main")["train"]
gsm8k = gsm8k_full.select(range(num_problems))

records = []

# ─── Helper to build the chat prompt ──────────────────────────────────────
def build_prompt(question: str) -> str:
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user",   "content": question}
    ]
    # apply_chat_template returns raw text with generation prompt appended
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# ─── Iterate and generate ─────────────────────────────────────────────────
for idx, example in enumerate(tqdm(gsm8k, desc="Collecting GSM8K chains")):
    
    q    = example["question"].strip()
    ref  = example["answer"].strip().split()[-1].strip()  # last token is the reference numeric answer
    prompt = build_prompt(q)

    # generate the full chain-of-thought + answer
    out = llm.generate(prompt, sampling_params)
    text = out[0].outputs[0].text

    # cap to MAX_CHAIN_TOKENS tokens (just in case)
    tokens = tokenizer.tokenize(text)
    if len(tokens) > MAX_CHAIN_TOKENS:
        tokens = tokens[:MAX_CHAIN_TOKENS]
        text   = tokenizer.convert_tokens_to_string(tokens)

    records.append({
        "idx": idx,
        "question": q,
        "reference_answer": ref,
        "chain_of_thought": text
    })

    # checkpoint every CHECKPOINT_EVERY examples
    if (idx + 1) % CHECKPOINT_EVERY == 0:
        df_ckpt = pd.DataFrame(records)
        ckpt_path = os.path.join(CHECKPOINT_DIR, f"ckpt_{idx+1:04d}.pkl")
        df_ckpt.to_pickle(ckpt_path)
        print(f"  ↪ saved checkpoint to {ckpt_path}")

# ─── Save final DataFrame ─────────────────────────────────────────────────
df = pd.DataFrame(records)
df.to_pickle(OUTPUT_PATH)
print(f"\n✓ Done! Saved full dataset to {OUTPUT_PATH}")

INFO 05-21 11:11:00 [__init__.py:239] Automatically detected platform cuda.
INFO 05-21 11:11:11 [config.py:717] This model supports multiple tasks: {'generate', 'score', 'reward', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 05-21 11:11:11 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-21 11:11:14 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reaso

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-21 11:11:17 [loader.py:458] Loading weights took 0.73 seconds
INFO 05-21 11:11:17 [gpu_model_runner.py:1347] Model loading took 3.3466 GiB and 2.353805 seconds
INFO 05-21 11:11:24 [backends.py:420] Using cache directory: /home/.cache/vllm/torch_compile_cache/7b5167d023/rank_0_0 for vLLM's torch.compile
INFO 05-21 11:11:24 [backends.py:430] Dynamo bytecode transform time: 6.71 s
INFO 05-21 11:11:30 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 5.278 s
INFO 05-21 11:11:31 [monitor.py:33] torch.compile takes 6.71 s in total
INFO 05-21 11:11:31 [kv_cache_utils.py:634] GPU KV cache size: 1,128,800 tokens
INFO 05-21 11:11:31 [kv_cache_utils.py:637] Maximum concurrency for 131,072 tokens per request: 8.61x
INFO 05-21 11:11:56 [gpu_model_runner.py:1686] Graph capturing finished in 24 secs, took 0.45 GiB
INFO 05-21 11:11:56 [core.py:159] init engine (profile, create kv cache, warmup model) took 38.43 seconds
INFO 05-21 11:11:56 [core_client.py

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  ↪ saved checkpoint to checkpoints/ckpt_1200.pkl

✓ Done! Saved full dataset to gsm8k_chains.pkl


In [15]:
import pandas as pd

# adjust path if necessary
trace_df = pd.read_pickle("gsm8k_chains.pkl")

# take a peek
print(trace_df.shape)
print(trace_df.head())

(1200, 4)
   idx                                           question reference_answer  \
0    0  Natalia sold clips to 48 of her friends in Apr...               72   
1    1  Weng earns $12 an hour for babysitting. Yester...               10   
2    2  Betty is saving money for a new wallet which c...                5   
3    3  Julie is reading a 120-page book. Yesterday, s...               42   
4    4  James writes a 3-page letter to 2 different fr...              624   

                                    chain_of_thought  
0  First, I identify the number of clips Natalia ...  
1  First, I need to calculate Weng's earnings bas...  
2  Okay, so Betty wants to buy a new wallet that ...  
3  To determine how many pages Julie should read ...  
4  To determine how many pages James writes in a ...  


In [19]:
token_lengths = [len(tokenizer(trace_df['chain_of_thought'].iloc[idx])['input_ids']) for idx in tqdm(range(len(trace_df)))]

  0%|          | 0/1200 [00:00<?, ?it/s]

In [21]:
trace_df['token_lengths'] = token_lengths


In [28]:
num_thoughts = [len(trace_df['chain_of_thought'].iloc[idx].split('\n\n')) for idx in tqdm(range(len(trace_df)))]

  0%|          | 0/1200 [00:00<?, ?it/s]

In [29]:
trace_df['num_thoughts'] = num_thoughts
trace_df.to_csv('reasoning_traces.csv')

In [33]:
trace_df['num_thoughts'].min()

3

In [34]:
trace_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   idx               1200 non-null   int64 
 1   question          1200 non-null   object
 2   reference_answer  1200 non-null   object
 3   chain_of_thought  1200 non-null   object
 4   token_lengths     1200 non-null   int64 
 5   num_thoughts      1200 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 56.4+ KB
